## Tokopedia Scrapper

### Dependencies

In [50]:
from datetime import datetime

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import time
import csv

### Global Variables

In [51]:
useragentarray = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
]

In [52]:
def get_url(search_term):
    """Generate an url from the search term"""
    template = "https://www.tokopedia.com/search?q={}"
    search_term = search_term.replace(' ', '+')

    url = template.format(search_term)
    url += '&page={}'

    return url

def scrape(keyword, start_page=1, end_page=2):
    request_url = get_url(keyword)

    options = Options()

    # Set up Chrome profile (bypass login using saved session)
    # chrome://version
    options.add_argument("--user-data-dir=C:/Users/Administrator/AppData/Local/Google/Chrome/User Data")
    options.add_argument("--profile-directory=Default")

    options.add_argument('--remote-debugging-port=9222')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')

    # Other options
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument('disable-notifications')
    options.add_argument('--disable-infobars')
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    options.add_experimental_option("prefs", {"profile.default_content_setting_values.notifications": 2})

    # Connect to the target tab using the session ID
    # debugger_address = target_tab['webSocketDebuggerUrl']
    driver = webdriver.Chrome(options=options)
    # driver.command_executor._url = debugger_address

    rows = []
    target_url = request_url
    idx = 1

    for page in range(start_page, end_page+1):

        # driver preparation
        driver.get(target_url.format(page))
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

        for i in range(len(useragentarray)):
            driver.execute_cdp_cmd(
                "Network.setUserAgentOverride", {"userAgent": useragentarray[i]}
            )
            # print(driver.execute_script("return navigator.userAgent;"))
            # driver.get("https://httpbin.io/headers")

        # scroll
        WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "css-5wh65g")))
        driver.execute_script("""
            var scroll = document.body.scrollHeight / 3;
            var i = 0;
            function scrollit(i) {
               window.scrollBy({top: scroll, left: 0, behavior: 'smooth'});
               i++;
               if (i < scroll) {
                setTimeout(scrollit, 500, i);
                }
            }
            scrollit(i);
            """)

        time.sleep(10)

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        # print(soup.prettify())

        for item in soup.find_all('div', {'class': 'css-5wh65g'}):
            link_tag = item.find('a', href=True)
            link = link_tag['href'] if link_tag else None

            name_tag = item.find('span', class_='_0T8-iGxMpV6NEsYEhwkqEg==')
            name = name_tag.text if name_tag else None

            price_tag = item.find('div', class_='_67d6E1xDKIzw+i2D2L0tjw==')
            price = price_tag.text if price_tag else None

            img_tag = item.find('img', alt='product-image')
            asset_url = img_tag['src'] if img_tag else None

            # # Debugging
            # print(f"Name: {name}")
            # print(f"Link: {link}")
            # print(f"Image URL: {asset_url}")
            # print(f"Price: {price}")
            # print('---')

            if not (name and price and asset_url and link):
                continue

            rows.append([idx, name, price, asset_url, link])
            idx = idx+1

    with open(f'tokopedia_items__{keyword}_{datetime.now().timestamp()}.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['no', 'name', 'price', 'asset_url', 'link'])
        writer.writerows(rows)

scrape('onitsuka', end_page=2)
scrape('air jordan', end_page=3)
